Estimate the binding fluxes using the methods in Hammes, G. G., Chang, Y.-C., & Oas, T. G. (2009). Conformational selection or induced fit: a flux description of reaction mechanism. Proceedings of the National Academy of Sciences of the United States of America, 106(33), 13737–13741. http://doi.org/10.1073/pnas.0907195106

In [1]:
def solve_free_protein(mdm2_tot,p53_tot,k_d_w,k_d_t,k_eq_wt,alpha=1.):
    k_d_w *= alpha
    k_d_t *= alpha
    a = 1./k_d_w+k_eq_wt/k_d_t
    b = 1.+k_eq_wt-mdm2_tot/k_d_w-k_eq_wt/k_d_t*mdm2_tot+p53_tot/k_d_w+p53_tot*k_eq_wt/k_d_t
    c = -(1.+k_eq_wt)*mdm2_tot
    print a,b,c
    mdm2_free1 = (-b+(b*b-4.*a*c)**0.5)/(2.*a)
    mdm2_free2 = (-b-(b*b-4.*a*c)**0.5)/(2.*a)
    return max(mdm2_free1,mdm2_free2)

In [12]:
def estimate_flux(p53_tot,k_wt,k_d_w,k_d_t,k_on_t,k_on_w,k_wt_l,mdm2_free):

    p53_w_unbound = p53_tot/(1+mdm2_free/k_d_w+k_eq_wt+k_eq_wt/k_d_t*mdm2_free)
    p53_t_unbound = k_eq_wt*p53_w_unbound
    p53_w_bound = p53_w_unbound/k_d_w*mdm2_free
    #p53_t_bound = k_eq_wt*p53_w_unbound/k_d_t*mdm2_free

    flux_cs = (1/(k_wt*p53_w_unbound)+1/(k_on_t*p53_t_unbound*mdm2_free))**-1
    flux_if = (1/(k_on_w*p53_w_unbound*mdm2_free+1/k_wt_l*p53_w_bound))**-1

    return flux_cs,flux_if

Rates and concentrations from the reference PNAS paper, Figure 3

In [3]:
#c_p53 = 20e-6
#k_d_w = 580./1e8
#k_d_t = 0.025/3e4
#k_eq_wt = 46./0.005

Rates and concentrations from our simulation. Rates are estimated using TPT

In [4]:
c_p53 = 7.1*10**-3 #Mol
# k_off_w/k_on_w = k_unfold_bound_to_unfold_unbound/k_unfold_unbound_to_unfold_bound
#                = 7.120e+06/1.537e+07
k_d_w = 7.120e+06/8.980e+09
# k_off_t/k_on_t = k_fold_bound_to_fold_unbound/k_fold_unbound_to_fold_bound
#                = 1.704e+06/6.565e+06
k_d_t = 1.704e+06/4.709e+10
# k_eq_wt = k_w_to_t/k_t_to_w = k_unfold_unbound_to_fold_unbound/k_fold_unbound_to_unfold_unbound
#                = 6.145e+05/2.162e+06
k_eq_wt = 6.145e+05/2.162e+06
#k_on_t = k_fold_unbound_to_fold_bound = 4.709e+10 1/(M*s)
k_on_t = 4.709e+10
#k_on_w = k_unfold_unbound_to_unfold_bound = 8.980e+09 1/(M*s)
k_on_w = 8.980e+09
#k_wt_l= k_w_to_t_bound = k_unfold_bound_to_fold_bound = 3.126e+06
k_wt_l = 3.126e+06
#k_wt = k_w_to_t_unbound = k_unfold_unbound_to_fold_unbound = 6.145e+05
k_wt = 6.145e+05

In [8]:
mdm2_free = solve_free_protein(mdm2_tot=c_p53, p53_tot=c_p53,
                         k_d_w=k_d_w, k_d_t=k_d_t,
                         k_eq_wt=k_eq_wt, alpha=1)
print mdm2_free

9115.85809896 1.28422756707 -0.00911801572618
0.000932156612795


In [14]:
print estimate_flux(p53_tot=c_p53,k_wt=k_wt
                    ,k_d_w=k_d_w,k_d_t=k_d_t
                    ,k_on_t=k_on_t,k_on_w=k_on_w
                    ,k_wt_l=k_wt_l,mdm2_free=mdm2_free)

(425.0972637910741, 6075.920995680314)


In [ ]:
import numpy as np

concentrations = np.exp(np.arange(-7,-2,0.5))

for c_p53 in np.exp(np.arange(-7,-2,0.5)):
    